In [3]:
#マスターキーとマスターチェーンコードの生成
import os
import binascii
import ecdsa
import hmac
import hashlib

# シード
seed = os.urandom(32)
# ルートキー
root_key = b"Bitcoin seed"

# ハッシュ関数「hmac-sha512」からハッシュ値を生成する関数
def hmac_sha512(data, keymessage):
    hash = hmac.new(data, keymessage, hashlib.sha512).digest()
    return hash

# 楕円曲線暗号と秘密鍵を利用して公開鍵を生成するための関数
def create_pubkey(private_key):
    publickey = ecdsa.SigningKey.from_string(private_key, curve = ecdsa.SECP256k1).verifying_key.to_string()
    return publickey

master = hmac_sha512(seed,root_key)
# マスター秘密鍵
master_secretkey = master[:32]
# マスターチェーンコード
master_chaincode = master[32:]
#　マスター公開鍵
master_publickey = create_pubkey(master_secretkey)

master_publickey_integer = int.from_bytes(master_publickey[32:], byteorder = "big")

# 圧縮公開鍵生成
if master_publickey_integer % 2 == 0:
    master_publickey_x = b"/x02" + master_publickey[:32]
else :
    master_publickey_x = b"/x03" + master_publickey[:32]
    
#　マスター秘密鍵
print("マスター秘密鍵")
print(binascii.hexlify(master_secretkey))
# マスターチェーンコード
print("マスターチェーンコード")
print(binascii.hexlify(master_chaincode))
# マスター圧縮公開鍵
print("マスター公開鍵")
print(binascii.hexlify(master_publickey_x))      

# 以下、子秘密鍵を生成するためのソースコード
index = 0
#インデックスを取得する
index_bytes = index.to_bytes(8, "big")
# 親秘密鍵とインデックスを結合してデータを作成する。
data = master_publickey_x + index_bytes
# ハッシュ化する。
result_hmac512 = hmac_sha512(data, master_chaincode)

# 親秘密鍵とHMAC512の結果の前半部分を足し合わせる
sum_integer = int.from_bytes(master_secretkey, "big") + int.from_bytes(result_hmac512[:32], "big")
# 巨大な素数
p = 2**256 - 2**9 -2**32 - 2**8 - 2**7 - 2**6 - 2**4 -1
# 子秘密鍵
child_secretkey = (sum_integer % p).to_bytes(32, "big")

#　子公開鍵
child_publickey = create_pubkey(child_secretkey)

child_publickey_integer = int.from_bytes(child_publickey[32:], byteorder = "big")

# 圧縮公開鍵生成
if child_publickey_integer % 2 == 0:
    child_publickey_x = b"/x02" + child_publickey[:32]
else :
    child_publickey_x = b"/x03" + child_publickey[:32]

# 子秘密鍵(マスターから見て1つ下の階層の秘密鍵)
print("子秘密鍵")
print(binascii.hexlify(child_secretkey))
# 子公開鍵
print("子公開鍵")
print(binascii.hexlify(child_publickey_x ))

# 子鍵からさらに子鍵を生成する。
#インデックスを取得する
index = 1
index_bytes = index.to_bytes(8, "big")
# ハッシュ化する。
result_hmac512 = hmac_sha512(index_bytes + child_publickey_x, child_secretkey[32:])
# 親秘密鍵とHMAC512の結果の前半部分を足し合わせる
sum_integer = int.from_bytes(child_secretkey, "big") + int.from_bytes(result_hmac512[:32], "big")
# 孫秘密鍵
child_secretkey2 = (sum_integer % p).to_bytes(32, "big")
#　子公開鍵
child_publickey2 = create_pubkey(child_secretkey2)

child_publickey_integer = int.from_bytes(child_publickey2[32:], byteorder = "big")

# 圧縮公開鍵生成
if child_publickey_integer % 2 == 0:
    child_publickey2_x = b"/x02" + child_publickey2[:32]
else :
    child_publickey2_x = b"/x03" + child_publickey2[:32]

# 孫秘密鍵(マスターから見て2つ下の階層の秘密鍵)
print("孫秘密鍵")
print(binascii.hexlify(child_secretkey2))
# 孫公開鍵
print("孫公開鍵")
print(binascii.hexlify(child_publickey2_x ))

マスター秘密鍵
b'e9c8659abcae41b3dcb0319db131626e7cf43c2da8734cc1c3231a1aa20862ac'
マスターチェーンコード
b'fc4d2b32aebfca22d13f08fcb59ce3a0ea71eea9bb6c1c154d690c25f5d38707'

マスター公開鍵
b'2f783032f60f41c8e22fa675ed269bdc0bf93e7bbd69b70037e4ab05c51400a7c242b511'
子秘密鍵
b'960132446614d77889c96da68d86cde180e611f03b72ac7c61c04eb2e9c7d6b8'
子公開鍵
b'2f7830325b95cf351322a9d39704e39d2985e0d8dd344dc61dedab6b4c75194724c9246f'
孫秘密鍵
b'bdd3ffd77bd9981186fc1b74cabc9909871ec59021d3e3f628a588db228a20d6'
孫公開鍵
b'2f7830324d5ca5eab9892e0f5289bddb06518f83f630ac6213104bccc98fb7264c2360c2'
